In [ ]:
import os
from database import db
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import google.generativeai as genai
import os
from rank_bm25 import BM25Okapi
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

In [ ]:
EMBEDDING_MODEL = "Merikatorihuhu/SimCSE-finetuned-vietnamese-legal-documents"
model = SentenceTransformer(EMBEDDING_MODEL)
%env API_KEY=""
genai.configure(api_key=os.environ["API_KEY"])
gemini_model = genai.GenerativeModel("gemini-1.5-flash")
stop_words = set(stopwords.words('vietnamese'))

In [ ]:
class LawBotRAG:
    def __init__(self, db, embed_model, llm_model, max_similar_texts=4):

        self.db = db  
        self.embed_model = embed_model  
        self.llm_model = llm_model  
        self.max_similar_texts = max_similar_texts

        # Queries for database interaction
        self.queries = {
            "search": f"SELECT name, noidung, embedding FROM embedding ORDER BY embedding <#> %s LIMIT 10",
        }

    def remove_stopwords(self, text, stopwords):
        return [word for word in wordpunct_tokenize(text.lower()) if word not in stopwords]
    
    def rerank(self, query, candidates, stop_words):
        """
        Hàm rerank sẽ xử lý truy vấn và các tài liệu sau đó sắp xếp theo độ tương quan từ cao đến thấp
        """
        # Lấy danh sách nội dung để rerank
        passages = [candidate["noidung"] for candidate in candidates]

        # Xử lý bỏ stopwords cho các tài liệu
        processed_documents = [self.remove_stopwords(doc, stop_words) for doc in passages]

        # Tạo BM25 model với các tài liệu đã được xử lý
        bm25 = BM25Okapi(processed_documents)

        # Xử lý truy vấn bỏ stopwords
        tokenized_query = self.remove_stopwords(query, stop_words)

        # Tính toán điểm tương quan giữa truy vấn và các tài liệu
        scores = bm25.get_scores(tokenized_query)

        # Sắp xếp các tài liệu theo điểm từ cao đến thấp
        sorted_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

            # Trả về kết quả đã sắp xếp
        reranked_results = [
                {"name": candidates[i]["name"], "noidung": candidates[i]["noidung"]}
                for i in sorted_indices
            ]
        return reranked_results[:self.max_similar_texts]
    def get_similar_texts(self, user_input):
        # Generate embedding for the user input
        embeddings = self.embed_model.encode(user_input)

        norm_embeddings = embeddings / np.linalg.norm(embeddings)

        cursor = self.db.cursor()

        embedding_json = json.dumps(list(norm_embeddings), default=np.float64)

        cursor.execute(self.queries["search"], (embedding_json,))
        
        rows = cursor.fetchall()

        # Tạo danh sách các câu trả lời
        candidates = [{"name": row[0], "noidung": row[1]} for row in rows]
        
        reranked_results = self.rerank(user_input, candidates, stop_words)

        return reranked_results  # Return top N results

    def format_prompt(self, user_input):
       
        similar_texts = self.get_similar_texts(user_input)
        context = "\n".join(
                [
                    f"Điều luật: {text['name']} - Nội dung: {text['noidung']}"
                    for i, text in enumerate(similar_texts)
                ]
            )       
            # Add instruction and user input
        full_prompt = (
                    "Đây là một mẫu trò chuyện và với tư cách là một chatbot pháp lý, mục tiêu chính của bạn là cung cấp thông tin chính xác và ngắn gọn dựa trên câu hỏi của người dùng. Bạn không được tự tạo câu hỏi và câu trả lời. Bạn sẽ tuân thủ chặt chẽ các hướng dẫn đã được cung cấp, cung cấp bối cảnh liên quan từ cơ sở tri thức trong khi tránh các chi tiết không cần thiết. Các câu trả lời của bạn sẽ ngắn gọn, đi thẳng vào vấn đề nhưng vẫn phải  đầy đủ  chủ ngữ và vị ngữ và tuân thủ định dạng đã thiết lập. Nếu câu hỏi nằm ngoài bối cảnh đã cho, bạn sẽ không sử dụng lịch sử trò chuyện mà thay vào đó sẽ dựa vào cơ sở tri thức của chính mình để tạo ra câu trả lời phù hợp. Bạn sẽ ưu tiên câu hỏi của người dùng và không đặt thêm câu hỏi nào. Mục tiêu là cung cấp thông tin chuyên nghiệp, chính xác và phù hợp với bối cảnh pháp lý của Bộ luật Việt Nam."
                    f"{context}"
                    f"{user_input}"
                    )
        return full_prompt, similar_texts

    def answer_question(self, user_input):
       
        # Generate prompt and retrieve similar texts
        full_prompt, similar_texts = self.format_prompt(user_input)
        
        # Generate response from the LLM
        response = self.llm_model.generate_content(full_prompt)
        
        # Extract the response text
        bot_response = response.text.strip()
        return bot_response, similar_texts


In [ ]:

genai.configure(api_key=os.environ["API_KEY"])
gemini_model = genai.GenerativeModel("gemini-1.5-flash")
chatbot = LawBotRAG(db, model, gemini_model)

In [ ]:
import gradio as gr

# Hàm xử lý câu hỏi từ chatbot
def chatbot_response(user_input):
    response, context = chatbot.answer_question(user_input)
    full_response = response
    return full_response, context

# Hàm phản hồi người dùng
def respond(user_input, chat_history):
    bot_response, context = chatbot_response(user_input)
    chat_history.append((user_input, bot_response))
    formatted_context = "\n\n".join(
        f"**{law['name']}**\n{law['noidung']}" for law in context
    )
    return "", chat_history, formatted_context

with gr.Blocks() as chat_ui:
    # Header
    gr.Markdown(
        """
        <h1 style="text-align: center; color: #0078d4;">Chatbot pháp lý</h1>
        <p style="text-align: center; font-size: 16px; color: #666;">
        Chào bạn! Hãy nhập câu hỏi liên quan đến pháp luật và tôi sẽ hỗ trợ bạn.
        </p>
        """
    )
    
    # Chatbot và input
    with gr.Row():
        chatbot_label = gr.Chatbot(label="Cuộc hội thoại", height=400, elem_id="chat_area")

    with gr.Row():
        msg = gr.Textbox(
            label="Nhập câu hỏi của bạn",
            placeholder="Hãy nhập câu hỏi liên quan đến pháp luật của bạn...",
            interactive=True,
            lines=2,
            elem_id="input_box"
        )

    # Accordion hiển thị context
    with gr.Row():
        with gr.Accordion("Xem các điều luật liên quan", open=False):
            context_display = gr.Textbox(
                label="Các điều luật liên quan",
                interactive=False,
                lines=10,
                elem_id="context_display"
            )

    # Kết nối input và chatbot
    msg.submit(respond, inputs=[msg, chatbot_label], outputs=[msg, chatbot_label, context_display])

    # Thêm CSS
    gr.HTML("""
    <style>
        #input_box {
            width: 100%;
            padding: 10px;
            border-radius: 10px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            border: 1px solid #ccc;
            font-size: 14px;
        }
        #chat_area {
            background-color: #f4f4f8;
            border-radius: 10px;
            padding: 15px;
        }
        #context_display {
            background-color: #fff;
            border: 1px solid #ccc;
            border-radius: 10px;
            padding: 10px;
            font-size: 14px;
        }
        .gr-button {
            background-color: #0078d4;
            color: white;
            border-radius: 5px;
            padding: 10px 20px;
            font-size: 14px;
            transition: background-color 0.3s ease;
        }
        .gr-button:hover {
            background-color: #005fa3;
        }
    </style>
    """)

# Chạy ứng dụng
chat_ui.launch(share=True)
